# Diseño de intercambiador de calor de tubo y coraza 

Jose Ramiro Fuentes Lara

In [11]:
import numpy as np

In [18]:
## Datos del sistema (Parámetros de inicialización)
"""Para los servicios se utiizará agua y agua de mar """

# flujo masico del agua 
m = 100000 

# Temperaturas
t1 = 25
t2 = 40

T1= 95
T2= 40

# Coeficiente global de transferencia de calor propuesto
Up = 1000


#Propiedades fisicas del agua
Cp = 4.2 # Calor específico
Mu = 0.8*10**-3 # Viscocidad
k = 0.59 # Coeficiente de trasferencia de calor 
rho = 1000 # Densidad

# Parámetros de diseño del intercambiador
do = 20 # Diametro exterior
t = 2 # Espesor del tubo 
di = 16 # Diametro interno
L = 4.88 # Longitud de un tubo

Np = 2 # numero de pasos totales por la coraza 


In [13]:
# Calculo del flujo de calor

def heat_flux(m,cp,T1,T2):
    Q = m*cp*abs(T2-T1)*(1/3600)
    return Q

Q = heat_flux(m,Cp,T1,T2)
print(f'Cantidad de calor transferido: {Q} KW')

Cantidad de calor transferido: 6416.666666666667 KW


In [17]:
# Calculo de diferencia de temperatura logaritmica 
def TempML(T1,T2,t1,t2):
    Tml = ((T1-t2)-(T2-t1))/np.log((T1-t2)/(T2-t1))
    return Tml

Tml = TempML(T1,T2,t1,t2)
print(f'Temperatura media logaritmica: {Tml} ºC')

Temperatura media logaritmica: 30.78621092446306 ºC


In [33]:
# Calculo cambio de temperatura real 

# Comenzamos con el calculo de los factores R y S
def FactorR(T1,T2,t1,t2):
    R = (T1-T2)/(t2-t1)
    return R

R = FactorR(T1,T2,t1,t2)

def FactorS(T1,T2,t1,t2):
    S = (t2-t1)/(T1-t1)
    return S

S = FactorS(T1,T2,t1,t2)

# Procedemos con el calculo del factor de corrección 
def CorrectionFT(R,S,Np):
    "Por facilidad interpretaremos la ecuacion por componentes"
    fact = (((R**2)+1)**(.5))
    
    top = fact*np.log((1-S)/(1-R*S))
    bottom = (R-1) * np.log( (Np-S*(R+1-fact)) / (Np-S*(R+1+fact) ))
    
    Ft = top/bottom
                           
    return Ft

Ft = CorrectionFT(R,S,Np)

# Para la temperatura real
Tr = Tml*Ft

print(f'Factor R = {R}')
print(f'Factor S = {S}')
print(f'Factor Ft = {Ft}')
print(f'Temperatura real = {Tr} ºC')

Factor R = 3.6666666666666665
Factor S = 0.21428571428571427
Factor Ft = 0.8121833326824699
Temperatura real = 25.00404738929587 ºC


In [40]:
# Area de transferencia de calor
def TotalA(Q,Up,Tr):
    "En esta ecuación hay una corrección de magnitud de *10^-3"
    At = (Q*10**3)/(Up*Tr)
    return At

At = TotalA(Q,Up,Tr)

print(f'Area total = {At} m^2') 

Area total = 256.6251201960854 m^2


In [45]:
# Para el area de un solo tubo 
def TubeArea(do,L):
    "En esta ecuación hay una corrección de magnitud de *10^-3"
    TbA = 2*np.pi*(do/2)*10**-3*L
    return TbA

TbA = TubeArea(do,L)
print(f'Area por un solo tubo = {TbA} m^2')

Area por un solo tubo = 0.3066194429903638 m^2


In [49]:
# Calculo del numero total de tubos 
NTb = At/TbA
print(f'Numero total de tubos = {NTb}')

Numero total de tubos = 836.9499262450568


In [54]:
#Calculo del flujo másico del segundo servicio 
def MassFlux(Q,Cp,t2,t1):
    m2 = Q/(Cp*(t2-t1))
    return  m2

m2 = MassFlux(Q,Cp,t2,t1)
print(f'Flujo masico 2 = {m2} Kg/s')

Flujo masico 2 = 101.85185185185186 Kg/s


In [56]:
#Flujo masico por tubo 
mTube = m2/(NTb/Np)
print(f'Flujo masico por tubo = {mTube} Kg/s')

Flujo masico por tubo = 0.2433881613654146 Kg/s


In [82]:
# velocidad del fluido en cada tubo 
def u_Tube(mTube,rho,di):
    "En esta ecuación hay una corrección de magnitud de *10^-3"
    A = (np.pi*(di*10**-3)**2)/4
    u_tube = mTube / (rho*A )
    return u_tube

ut = u_Tube(mTube,rho,di)

print(f'Velocidad en cada tubo= {ut} m/s')

Velocidad en cada tubo= 1.2105134053548001 m/s


In [86]:
# Numeros adimensionales para los tubos 
def Reynolds(rho,u,di,Mu):
    "En esta ecuación hay una corrección de magnitud de *10^-3"
    Re = rho*u*(di*10**-3)/Mu
    return Re

Tube_Re = Reynolds(rho,ut,di,Mu)

print(f'Reynolds en los tubos = {Tube_Re}')

Reynolds en los tubos = 24210.268107096002


In [141]:
def Prandtl(Cp,Mu,k):
    "En esta ecuación hay una corrección de magnitud de *10^3"
    Pr = Cp*10**3*Mu/k
    return Pr

Pr = Prandtl(Cp,Mu,k)

print(f'Valor de Prandtl = {Pr}')

Valor de Prandtl = 5.694915254237289


In [144]:
# Para determinar el coeficiente de trasferencia de calor para los tubos 
"Se hara uso de un diagrama el cual se espera posteriormente encontrar su polinomio 12.23"

JH = 3.8*10**-3

def hT_coefficient(JH,Re,Pr,kf,di):
    hT = ( JH * Re * ((Pr)**(1/3)) * kf ) / (di*10**-3)
    return hT

hT = hT_coefficient(JH,Tube_Re,Pr,k,di)
print( f"Coeficiente de tranferencia de calor para tubos = {hT_coefficient(JH,Tube_Re,Pr,k,di)} W/m^2ºC")

Coeficiente de tranferencia de calor para tubos = 6058.20985442534 W/m^2ºC


In [153]:
# Caida de presion para los tubos 
"Se hara uso de un diagrama el cual se espera posteriormente encontrar su polinomio 12.24"

JF = 3.8*10**-3

def pressure_drop(Np,JF,L,di,rho,u):
    DP = Np * (8*JF*(L/(di*(10**-3)))+2.5)*((rho*(u**2))/2) 
    return DP

DP = pressure_drop(Np,JF,L,di,rho,ut)
print(f'Caida de presion en los tubos = {DP} Pa')




Caida de presion en los tubos = 17250.014317888137 Pa


In [202]:
# Para determinar el coeficiente de trasferencia de calor para la coraza y la caida de presion
# se necesitan calcular unos datos antes

# Pitch
Pt = 1.25*do*10**-3
print(f'Pitch = {Pt} m')

# Diametro del banco de tubos
"Se usaran datos obtenidos de tablas"
k1 = .249
m1 = 2.207

Db = (do*10**-3)*(NTb/k1)**(1/m1)
print(f'Diametro de banco de tubos={Db} m')

# Diametro interno de la coraza
"De la grafica bundle diameter vs shell inside se obtiene un valor de BD"
BD = 65*10**-3

Ds = Db+BD
print(f'Diametro Coraza = {Ds} m')

# Separacion de los bafles
Lb = 0.4*Ds
print(f'Separacion de baffles = {Lb} m')

# Area de la coraza
def As_shell(Pt,do,Ds,Lb):
    As = (Pt-(do*10**-3))*Ds*Lb/Pt
    return As

As = As_shell(Pt,do,Ds,Lb)

print(f'Area de la coraza = {As} m^2')

# Velocidad masica
"Conversion de horas a segundos en el flujo masico"
Gs = m*(1/3600)/As
print(f'Velocidad másica = {Gs} Kg/m^2s')

# Velocidad lineal coraza
us = Gs/rho
print(f"Velocidad lineal coraza = {us} m/s")

# Diametro equivalente para el arregl triangular
de = ( 1.10 / (do*10**-3) ) * ((Pt**2)-(0.917*(do*10**-3)**2))
print(f"Diametro equivalente = {de}")

# Calculo del reynolds para coraza
ReS = (Gs*de)/Mu
print(f"Reynolds coraza = {ReS}")

# Calculo del Prandtl Coraza
PrS = Cp*10**3*Mu/k
print(f"Prandtl coraza = {PrS}")

Pitch = 0.025 m
Diametro de banco de tubos=0.792319721724648 m
Diametro Coraza = 0.857319721724648 m
Separacion de baffles = 0.34292788868985924 m
Area de la coraza = 0.058799768420642246 m^2
Velocidad másica = 472.4130472599976 Kg/m^2s
Velocidad lineal coraza = 0.4724130472599976 m/s
Diametro equivalente = 0.014201000000000005
Reynolds coraza = 8385.922105174035
Prandtl coraza = 5.694915254237289


In [203]:
# Para determinar el coeficiente de trasferencia de calor para la coraza
"Usando constantes dadas en tablas JH y JF"
JHs = 6.5*10**-3
JFs = 5*10**-2

hs = JHs*ReS*(PrS**(1/3))*k/de
print(f"Coeficiente de transferencia de calor coraza = {hs} W/m^2ºC")

Coeficiente de transferencia de calor coraza = 4044.1416006372033 W/m^2ºC


In [211]:
# Caida de presion para la coraza
DPs = 8 * JFs * (Ds/de) * (L/Lb) * ((rho*(us**2)) / 2)
print(f"Caida de presion en la coraza = {DPs} Pa")

Caida de presion en la coraza = 38345.52304910943 Pa


In [231]:
# Calculo del coeficiente global de transderencia de calor 
"Se obtienen los valores del factor de ensuciamiento y conductividad termica"
had = 5000
hid = 5000
K = 50
Denominador = (1/hs) + (1/had) + ((do/1000)*np.log(do/di)/(2 * K)) + (((do)/(di))*(1/hid)) + ((do/di)*(1/hT))

U_Global = 1/Denominador

print(f"Coeficiente global de transferencia de calor: {U_Global} W/m^2ºC")

Coeficiente global de transferencia de calor: 1054.5947336010315 W/m^2ºC
